In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing

from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.tag import pos_tag, pos_tag_sents
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

import re
import string

## Loading and Preliminary Data Cleaning and EDA

In [2]:
path = r'C:\Users\nesha\Desktop\movies_metadata.csv' # data path
data = pd.read_csv(path)
    
data.shape

C:\Users\nesha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(45466, 24)

In [3]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

The main column of interest for this project is "overview". Let's take a look at some samples to get an idea of the formatting of the plots!

In [5]:
data.overview[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [6]:
data.overview[1]

"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."

In [7]:
data.overview[5]

'Obsessive master thief, Neil McCauley leads a top-notch crew on various insane heists throughout Los Angeles while a mentally unstable detective, Vincent Hanna pursues him without rest. Each man recognizes and respects the ability and the dedication of the other even though they are aware their cat-and-mouse game may end in violence.'

In [8]:
#looking into overview length statistics

print("Min Overview Words", data.overview.str.split().str.len().min())
print("Max Overview Words", data.overview.str.split().str.len().max())
print("Average Overview Words", round(data.overview.str.split().str.len().mean(),0))
print("Overview Words Standard Deviation", round(data.overview.str.split().str.len().std(),0))

Min Overview Words 0.0
Max Overview Words 187.0
Average Overview Words 55.0
Overview Words Standard Deviation 35.0


In [9]:
#looking for number of super short overviews - 10 words or less
mask = (data.overview.str.split().str.len() <= 10)
data.loc[mask]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
40,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",NaN,31174,tt0114279,en,Richard III,Shakespeare's Play transplanted into a 1930s s...,...,1995-12-29,0.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"I can smile, and murder while I smile",Richard III,False,6.9,50.0
104,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,11907,tt0110251,de,Keiner liebt mich,No overview found.,...,1994-11-07,0.0,104.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,Nobody Loves Me,False,5.1,5.0
137,False,NaN,0,[],NaN,124639,tt0114618,en,Target,A subtle yet violent commentary on feudal lords.,...,1995-08-01,0.0,122.0,[],Released,NaN,Target,False,0.0,0.0
553,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,161158,tt0107779,en,"Paris, France",A writer has torrid fantasy affairs with young...,...,1994-02-04,0.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,"Paris, France",False,6.0,2.0
566,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,95743,tt0109828,en,Foreign Student,A French football playing exchange student fal...,...,1994-07-29,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Foreign Student,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45430,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...",NaN,222872,tt0120200,en,Starquest II,Sci-fi thriller directed by Fred Gallo.,...,1996-01-01,0.0,0.0,[],Released,NaN,Starquest II,False,2.0,1.0
45433,False,NaN,0,"[{'id': 10402, 'name': 'Music'}, {'id': 35, 'n...",NaN,5589,tt0232750,cs,Šíleně smutná princezna,No overview found.,...,1968-06-07,0.0,73.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Released,NaN,Šíleně smutná princezna,False,6.1,4.0
45445,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...",NaN,49277,tt0135122,en,Les affiches en goguette,A wall full of advertising posters comes to life.,...,1906-01-01,0.0,3.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,The Hilarious Posters,False,4.5,2.0
45451,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,106807,tt0135571,fr,Nouvelles luttes extravagantes,A series of fantastical wrestling matches.,...,1900-01-01,0.0,2.0,"[{'iso_639_1': 'xx', 'name': 'No Language'}]",Released,NaN,The Fat and Lean Wrestling Match,False,6.5,6.0


Seems like most of the information are missing for these movies. Also some have "No overview found" as their overview!! Let's consider overviews less than 20 as too short and drop these movies!

In [10]:
#dropping overviews with 20 words or fewer 
data.drop(data[data['overview'].str.split().str.len() <= 20].index, inplace=True)

In [11]:
data.shape

(39214, 24)

Let's look into the overview length statistics once more:

In [12]:
#looking into overview length statistics again!!

print("Min Overview Words", data.overview.str.split().str.len().min())
print("Max Overview Words", data.overview.str.split().str.len().max())
print("Average Overview Words", round(data.overview.str.split().str.len().mean(),0))
print("Overview Words Standard Deviation", round(data.overview.str.split().str.len().std(),0))

Min Overview Words 21.0
Max Overview Words 187.0
Average Overview Words 62.0
Overview Words Standard Deviation 33.0


In [13]:
data.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [14]:
#looking for null values
data.isnull().sum()

adult                        0
belongs_to_collection    35135
budget                       0
genres                       0
homepage                 32139
id                           0
imdb_id                     16
original_language            8
original_title               0
overview                   954
popularity                   1
poster_path                269
production_companies         1
production_countries         1
release_date                70
revenue                      1
runtime                    258
spoken_languages             1
status                      68
tagline                  21327
title                        1
video                        1
vote_average                 1
vote_count                   1
dtype: int64

We have 954 null overviews that we need to drop!!

In [15]:
#dropping null overviews
data.drop(data[data['overview'].isnull()].index, inplace=True)

In [16]:
data.shape

(38260, 24)

In [17]:
#looking for null values
data.overview.isnull().sum()

0

In [18]:
#looking for duplicates
data.duplicated(subset=None, keep=False).sum()

23

In [19]:
#looking for duplicate overviews
data.duplicated(subset=['overview'], keep=False).sum()

90

In [20]:
data.nunique(axis=0)

adult                        2
belongs_to_collection     1619
budget                    1117
genres                    3801
homepage                  6962
id                       38233
imdb_id                  38219
original_language           86
original_title           36678
overview                 38214
popularity               37388
poster_path              37988
production_companies     19893
production_countries      2145
release_date             15660
revenue                   6248
runtime                    339
spoken_languages          1798
status                       6
tagline                  17772
title                    35818
video                        2
vote_average                92
vote_count                1773
dtype: int64

In [21]:
#There are more unique overviews than there are titles meaning same movie titles must have different overviews?!
#dropping duplicate movies based on their title and runtime
data.drop_duplicates(subset=['title', 'runtime'], keep='last', inplace=True)

In [22]:
#looking for duplicate overviews once more
data.duplicated(subset=['overview'], keep=False).sum()

38

In [23]:
#dropping the rest of overview duplicates
data.drop_duplicates(subset=['overview'], keep='last', inplace=True)
data.shape

(38145, 24)

In [24]:
#going to keep the title, release_date, and overview
data.drop(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'video',
       'vote_average', 'vote_count'], axis=1, inplace=True)

In [25]:
data.shape
data.head()

,overview,release_date,title
0,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,Toy Story
1,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji
2,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men
3,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,Waiting to Exhale
4,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II


In [26]:
#saving the dataset
data.to_csv(r'C:\Users\nesha\Desktop\movies.csv', index = False)

## Text Cleaning and Preprocessing

In [27]:
#removing numbers and punctuations

#remove numbers
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

#remove punctuation
punct = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)

data['overview'] = data.overview.map(alphanumeric).map(punct)

In [28]:
data.overview[0]

'Led by Woody  Andy s toys live happily in his room until Andy s birthday brings Buzz Lightyear onto the scene  Afraid of losing his place in Andy s heart  Woody plots against Buzz  But when circumstances separate Buzz and Woody from their owner  the duo eventually learns to put aside their differences '

In [29]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nesha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
#Preprocessing: Tokenizing
data['overview_token'] = data['overview'].apply(word_tokenize)

In [31]:
#let's look at a sample
data.overview_token[0]

['Led',
 'by',
 'Woody',
 'Andy',
 's',
 'toys',
 'live',
 'happily',
 'in',
 'his',
 'room',
 'until',
 'Andy',
 's',
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene',
 'Afraid',
 'of',
 'losing',
 'his',
 'place',
 'in',
 'Andy',
 's',
 'heart',
 'Woody',
 'plots',
 'against',
 'Buzz',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner',
 'the',
 'duo',
 'eventually',
 'learns',
 'to',
 'put',
 'aside',
 'their',
 'differences']

In [32]:
nltk.download('averaged_perceptron_tagger')

#Preprocessing: Parts of Speech Tagging
#labeling each word as noun,verbs,adjective, etc.
data['overview_tag'] = pos_tag_sents(data['overview_token'].tolist()) 

#let's look at a sample
data.overview_tag[0]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nesha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Led', 'VBN'),
 ('by', 'IN'),
 ('Woody', 'NNP'),
 ('Andy', 'NNP'),
 ('s', 'VBD'),
 ('toys', 'NNS'),
 ('live', 'VBP'),
 ('happily', 'RB'),
 ('in', 'IN'),
 ('his', 'PRP$'),
 ('room', 'NN'),
 ('until', 'IN'),
 ('Andy', 'NNP'),
 ('s', 'VBD'),
 ('birthday', 'JJ'),
 ('brings', 'NNS'),
 ('Buzz', 'NNP'),
 ('Lightyear', 'NNP'),
 ('onto', 'IN'),
 ('the', 'DT'),
 ('scene', 'NN'),
 ('Afraid', 'NNP'),
 ('of', 'IN'),
 ('losing', 'VBG'),
 ('his', 'PRP$'),
 ('place', 'NN'),
 ('in', 'IN'),
 ('Andy', 'NNP'),
 ('s', 'JJ'),
 ('heart', 'NN'),
 ('Woody', 'NNP'),
 ('plots', 'NNS'),
 ('against', 'IN'),
 ('Buzz', 'NNP'),
 ('But', 'CC'),
 ('when', 'WRB'),
 ('circumstances', 'NNS'),
 ('separate', 'VBP'),
 ('Buzz', 'NNP'),
 ('and', 'CC'),
 ('Woody', 'NNP'),
 ('from', 'IN'),
 ('their', 'PRP$'),
 ('owner', 'NN'),
 ('the', 'DT'),
 ('duo', 'NN'),
 ('eventually', 'RB'),
 ('learns', 'VBZ'),
 ('to', 'TO'),
 ('put', 'VB'),
 ('aside', 'RP'),
 ('their', 'PRP$'),
 ('differences', 'NNS')]

In [33]:
 nltk.download('tagsets')
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\nesha\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [34]:
#creating a function to remove proper nouns
def remove_pn(tagged_text):
    return [(word, tag) for word,tag in tagged_text if tag != 'NNP' and tag != 'NNPS' and tag != 'IN' and tag != 'TO' 
           and tag != 'DT' and tag != 'CC' and tag != 'VBD' and tag != 'PRP$' and tag != 'VBZ']

#applying the function
data['overview_tagged_pn_removed'] = data['overview_tag'].apply(remove_pn)

In [35]:
#Preprocessing: Lemmatizing

#creating a function to use pos_tags to lemmatize overviews using WordNetLemmatizer
def lemmatize_pos(tagged_text):
    list_lemm = []
    wnl = WordNetLemmatizer()
    for word, tag in (tagged_text):
        if tag.startswith("NN"): #noun, common, singular or mass
            list_lemm.append(wnl.lemmatize(word, wordnet.NOUN))
        elif tag.startswith('VB'): #verb, base form
            list_lemm.append(wnl.lemmatize(word, wordnet.VERB))
        elif tag.startswith('JJ'): #adjective or numeral, ordinal
            list_lemm.append(wnl.lemmatize(word, wordnet.ADJ))
        else:
            list_lemm.append(word)
    return list_lemm

In [36]:
nltk.download('wordnet')

#applying thefunction
data['overview_lemm'] = data['overview_tagged_pn_removed'].apply(lemmatize_pos)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nesha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
#looking at a sample
data['overview_lemm'][0]

['Led',
 'toy',
 'live',
 'happily',
 'room',
 'birthday',
 'brings',
 'scene',
 'lose',
 'place',
 's',
 'heart',
 'plot',
 'when',
 'circumstance',
 'separate',
 'owner',
 'duo',
 'eventually',
 'put',
 'aside',
 'difference']

Seems that we've successfully removed the proper nouns and the part of speech tags!

In [38]:
data.head()

,overview,release_date,title,overview_token,overview_tag,overview_tagged_pn_removed,overview_lemm
0,Led by Woody Andy s toys live happily in his ...,1995-10-30,Toy Story,"[Led, by, Woody, Andy, s, toys, live, happily,...","[(Led, VBN), (by, IN), (Woody, NNP), (Andy, NN...","[(Led, VBN), (toys, NNS), (live, VBP), (happil...","[Led, toy, live, happily, room, birthday, brin..."
1,When siblings Judy and Peter discover an encha...,1995-12-15,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[(When, WRB), (siblings, NNS), (Judy, NNP), (a...","[(When, WRB), (siblings, NNS), (discover, VBP)...","[When, sibling, discover, enchant, board, game..."
2,A family wedding reignites the ancient feud be...,1995-12-22,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[(A, DT), (family, NN), (wedding, NN), (reigni...","[(family, NN), (wedding, NN), (ancient, JJ), (...","[family, wedding, ancient, feud, next, door, n..."
3,Cheated on mistreated and stepped on the wom...,1995-12-22,Waiting to Exhale,"[Cheated, on, mistreated, and, stepped, on, th...","[(Cheated, VBN), (on, IN), (mistreated, VBN), ...","[(Cheated, VBN), (mistreated, VBN), (stepped, ...","[Cheated, mistreat, step, woman, be, hold, bre..."
4,Just when George Banks has recovered from his ...,1995-02-10,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...","[(Just, RB), (when, WRB), (George, NNP), (Bank...","[(Just, RB), (when, WRB), (recovered, VBN), (d...","[Just, when, recover, daughter, s, wedding, he..."


In [39]:
#droppinh columns we wont need in the upcoming steps
data.drop(['overview','overview_token','overview_tag', 'overview_tagged_pn_removed'], axis=1, inplace=True)

In [40]:
data.head()

,release_date,title,overview_lemm
0,1995-10-30,Toy Story,"[Led, toy, live, happily, room, birthday, brin..."
1,1995-12-15,Jumanji,"[When, sibling, discover, enchant, board, game..."
2,1995-12-22,Grumpier Old Men,"[family, wedding, ancient, feud, next, door, n..."
3,1995-12-22,Waiting to Exhale,"[Cheated, mistreat, step, woman, be, hold, bre..."
4,1995-02-10,Father of the Bride Part II,"[Just, when, recover, daughter, s, wedding, he..."


In [41]:
data.shape

(38145, 3)

In [43]:
#saving the preprocessed data
data.to_csv(r'C:\Users\nesha\Desktop\movies_preprocessed1.csv', index = False)